In [1]:

import scipy.sparse as sp
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import re
import numpy as np

from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2


# natural language toolkit
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.tag import pos_tag_sents
from nltk.stem import WordNetLemmatizer

# SciKit-Learn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from skopt import gp_minimize, forest_minimize, dummy_minimize

import pandas as pd

train = pd.read_csv('train.csv')


test = pd.read_csv('test.csv')


#train



In [2]:
#preprocessing


comment_data = train

#clean
comment_data['prep'] = comment_data['comment'].str.replace(r'[^\w\s]+', '') # tira td que nao palavra e espaco em branco
comment_data['prep'] = comment_data['prep'].str.lower()
comment_data['prep'] = comment_data['prep'].str.replace('(\d+)', ' num ')
comment_data['prep'] = comment_data['prep'].str.replace(r'http(?<=http).*', ' ')
comment_data['prep'] = comment_data['prep'].str.replace(r'\s+', " ")
comment_data['prep'] = comment_data['prep'].str.replace(" +", " ")

#comment_data





test_data = test
#clean
test_data['prep'] = test_data['comment'].str.replace(r'[^\w\s]+', '')
test_data['prep'] = test_data['prep'].str.lower()
test_data['prep'] = test_data['prep'].str.replace('(\d+)', ' num ')
test_data['prep'] = test_data['prep'].str.replace(r'http(?<=http).*', ' ')
test_data['prep'] = test_data['prep'].str.replace(r'\s+', " ")
test_data['prep'] = test_data['prep'].str.replace(" +", " ")

In [3]:
lemmatizer = WordNetLemmatizer()
tt = TweetTokenizer()
def lemmatize_col(row):
    row = tt.tokenize(row)
    return ' '.join([lemmatizer.lemmatize(w) for w in row])

comment_data['prep'] = comment_data['prep'].apply(lemmatize_col)
test_data['prep'] = test_data['prep'].apply(lemmatize_col)

# stopwords
stop = stopwords.words('english')
comment_data['prep'] = comment_data['prep'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
test_data['prep'] = test_data['prep'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [4]:
clean_data = comment_data['prep'].to_numpy()
clean_labels = comment_data['subreddit'].to_numpy()


In [5]:
#com pre proc

clean_data
clean_labels

clean_test=test_data['prep']
#features_train

### com pre proc


In [6]:
def report(results, top=3):
    for i in range(1, top+1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})"
                  .format(results['mean_test_score'][candidate],
                          results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [7]:



#Logistic Regression
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.preprocessing import Normalizer
norm=Normalizer()
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import time


param_dist = {'clf__penalty': ['l2'],
              'clf__solver': ['liblinear'],
              'clf__C': np.linspace(0.1,5,5)}
clf = LogisticRegression(multi_class='auto')
text_clf = Pipeline([('vect',CountVectorizer(stop_words='english', ngram_range=(1,2), min_df = 10, max_df = 1.)),('tfidf',TfidfTransformer()),('clf',clf)])
random_search = GridSearchCV(text_clf, param_grid = param_dist, cv=10)
start_time = time.time()
random_search.fit(clean_data, clean_labels)
print("GridSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time.time() - start_time), len(random_search.cv_results_['params'])))
report(random_search.cv_results_)

GridSearchCV took 1462.70 seconds for 5 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.455 (std: 0.005)
Parameters: {'clf__C': 1.3250000000000002, 'clf__penalty': 'l2', 'clf__solver': 'liblinear'}

Model with rank: 2
Mean validation score: 0.453 (std: 0.005)
Parameters: {'clf__C': 2.5500000000000003, 'clf__penalty': 'l2', 'clf__solver': 'liblinear'}

Model with rank: 3
Mean validation score: 0.450 (std: 0.006)
Parameters: {'clf__C': 3.7750000000000004, 'clf__penalty': 'l2', 'clf__solver': 'liblinear'}



In [ ]:
print("GridSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time.time() - start_time), len(random_search.cv_results_['params'])))

In [ ]:
report(random_search.cv_results_)

In [6]:
#Logistic Regression
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.preprocessing import Normalizer
norm=Normalizer()
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression




param_dist = {'clf__penalty': ['l2'],
              'clf__solver': ['liblinear'],
              'clf__C': np.linspace(0.1,5,5)}


pipeline_LogisticRegression = Pipeline([
                     ('vect', CountVectorizer()),
                     ('chi',  SelectKBest(chi2, k=10000)),
                     ('tfidf', TfidfTransformer()),
                     ('norm', Normalizer()),
                     ('clf', LogisticRegression(multi_class='auto', random_state=0))
])

grid_search = GridSearchCV(pipeline_LogisticRegression, {}, n_jobs=-1, verbose=1, cv=10, scoring='accuracy')
grid_search.fit(clean_data, clean_labels)
print(grid_search.best_score_)
print(grid_search.cv_results_)

previsoes_test_pre_proc=grid_search.predict(clean_test)



Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  2.9min finished
/home/gabriel/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.47614999999999996
{'mean_fit_time': array([61.58336327]), 'std_fit_time': array([15.70865445]), 'mean_score_time': array([0.43635411]), 'std_score_time': array([0.12109717]), 'params': [{}], 'split0_test_score': array([0.47316667]), 'split1_test_score': array([0.47366667]), 'split2_test_score': array([0.47516667]), 'split3_test_score': array([0.4695]), 'split4_test_score': array([0.4705]), 'split5_test_score': array([0.47633333]), 'split6_test_score': array([0.478]), 'split7_test_score': array([0.48883333]), 'split8_test_score': array([0.4785]), 'split9_test_score': array([0.47783333]), 'mean_test_score': array([0.47615]), 'std_test_score': array([0.00514644]), 'rank_test_score': array([1], dtype=int32)}


In [7]:
'''
cv=5 ...0.4705333333333333

cv=10... 0.47614999999999996'''

'\ncv=5 ...0.4705333333333333\n\ncv=10... 0.47614999999999996'

In [8]:
#SEM PRE PROC


train = pd.read_csv('train.csv')
#print(train)

test = pd.read_csv('test.csv')


features_train=train['comment']

target_train=train['subreddit']



features_test=test['comment']


features_train
#target_train

#test.head(20)

0        I think prestige points should not expire ever...
1        Whats going to happen with them if they will b...
2        Anecdotal evidence is anecdotal. Clearly by “e...
3        Look dude, with all due respect, your music is...
4                        Hope he gets the doomhammer back!
                               ...                        
59995                         Yo this guy Luka pretty good
59996                        Unplug these things right now
59997    Well said. Do you think they’ll resonate with ...
59998    So we can impeach a president for lying? Pleas...
59999    Too broad dude, get ready for the shit my pant...
Name: comment, Length: 60000, dtype: object

###### sem preproc

In [9]:
#Logistic Regression
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression


pipeline_LogisticRegression = Pipeline([
                     ('vect', CountVectorizer()),
                     ('chi',  SelectKBest(chi2, k=10000)),
                     ('tfidf', TfidfTransformer()),
                     ('norm', Normalizer()),
                     ('clf', LogisticRegression(random_state=0))
])

grid_search = GridSearchCV(pipeline_LogisticRegression, {}, n_jobs=-1, verbose=1, cv=10, scoring='accuracy')
grid_search.fit(features_train, target_train)
print(grid_search.best_score_)
print(grid_search.cv_results_)

previsoes_test=grid_search.predict(features_test)



Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  3.4min finished
/home/gabriel/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.47783333333333344
{'mean_fit_time': array([71.05292385]), 'std_fit_time': array([14.59525171]), 'mean_score_time': array([0.76388927]), 'std_score_time': array([0.24300821]), 'params': [{}], 'split0_test_score': array([0.4805]), 'split1_test_score': array([0.48266667]), 'split2_test_score': array([0.475]), 'split3_test_score': array([0.4595]), 'split4_test_score': array([0.47016667]), 'split5_test_score': array([0.48033333]), 'split6_test_score': array([0.48316667]), 'split7_test_score': array([0.49133333]), 'split8_test_score': array([0.47566667]), 'split9_test_score': array([0.48]), 'mean_test_score': array([0.47783333]), 'std_test_score': array([0.00812233]), 'rank_test_score': array([1], dtype=int32)}


In [10]:
'''
results cv=5 ...0.4705333333333333

results cv=10 ... 0.47783333333333344'''

'\nresults cv=5 ...0.4705333333333333\n\nresults cv=10 ... 0.47783333333333344'

In [11]:
'''#gerar arquivo 

previsoes_test = pd.DataFrame(previsoes_test, columns=['Category'])

previsoes_test['Id'] = previsoes_test.index

previsoes_test = previsoes_test.reindex(columns=['Id', 'Category'])

previsoes_test.to_csv('logistic_regression_17jul.csv', index=False)'''




"#gerar arquivo \n\nprevisoes_test = pd.DataFrame(previsoes_test, columns=['Category'])\n\nprevisoes_test['Id'] = previsoes_test.index\n\nprevisoes_test = previsoes_test.reindex(columns=['Id', 'Category'])\n\nprevisoes_test.to_csv('logistic_regression_17jul.csv', index=False)"